First, we import all the necessary libraries needed and store the dataset

In [22]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [23]:
news_data=pd.read_csv('Combined_News_DJIA.csv')

In [24]:
news_data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

Now, we create the training and testing datasets as shown below

In [25]:
train = news_data[news_data['Date'] < '2014-01-01']
test = news_data[news_data['Date'] > '2013-12-31']

Processing texts in different ways

In [26]:
example = train.iloc[3,8]
print(example)

b"Bush announces Operation Get All Up In Russia's Grill. Yeah, this will end well."


In [27]:
print(example.upper())

B"BUSH ANNOUNCES OPERATION GET ALL UP IN RUSSIA'S GRILL. YEAH, THIS WILL END WELL."


In [28]:
example3 = CountVectorizer().build_tokenizer()(example.upper())
print(example3)

['BUSH', 'ANNOUNCES', 'OPERATION', 'GET', 'ALL', 'UP', 'IN', 'RUSSIA', 'GRILL', 'YEAH', 'THIS', 'WILL', 'END', 'WELL']


In [29]:
pd.DataFrame([[x,example3.count(x)] for x in set(example3)], columns = ['Words', 'Count'])

,Words,Count
0,BUSH,1
1,WELL,1
2,ANNOUNCES,1
3,GET,1
4,UP,1
5,IN,1
6,YEAH,1
7,WILL,1
8,END,1
9,RUSSIA,1


Now, we append all the headlines of each day into a single string for easy accessibility

In [30]:
train_set = []
for row in range(0,len(train.index)):
    train_set.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [31]:
print(train_set[0])

b"Georgia 'downs two Russian warplanes' as countries move to brink of war" b'BREAKING: Musharraf to be impeached.' b'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)' b'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire' b"Afghan children raped with 'impunity,' U.N. official says - this is sick, a three year old was raped and they do nothing" b'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.' b"Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO's side" b"The 'enemy combatent' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." b'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]' b'Did the U.S. Prep Georgia for War with Russia?' b'Rice Gives Green 

This step creates a model based on the count of words of the training set

In [32]:
basicvectorizer = CountVectorizer()
basictrain = basicvectorizer.fit_transform(train_set)
print(basictrain.shape)

(1359, 29493)


In [33]:
basicmodel = LogisticRegression(solver='lbfgs', max_iter=1000)
basicmodel = basicmodel.fit(basictrain, train["Label"])

Since the model has been trained, we now create the testing set in the same way

In [34]:
test_set = []
for row in range(0,len(test.index)):
    test_set.append(' '.join(str(x) for x in test.iloc[row,2:27]))
basictest = basicvectorizer.transform(test_set)
predictions = basicmodel.predict(basictest)


After storing predictions, we use crosstab to view the accuracy of the model

In [35]:
pd.crosstab(test["Label"], predictions, rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,119,180
1,142,189


We see that the accuracy is 48 % for the predicted values.
We can also view the coefficients of our model and compare it with the words to see the most affecting of them all.

In [36]:
basicwords = basicvectorizer.get_feature_names()
basiccoeffs = basicmodel.coef_.tolist()[0]
coeffdf = pd.DataFrame({'Word' : basicwords, 
                        'Coefficient' : basiccoeffs})
coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
coeffdf.head(10)

,Word,Coefficient
26512,three,0.452257
23506,self,0.450911
24497,so,0.432908
10127,fears,0.430386
23873,ship,0.407403
14733,kills,0.397041
5990,company,0.378402
16932,missing,0.376203
2110,another,0.372866
23492,seize,0.372011


These values seem correct as words like "kill" and "missing" have high impact on stock prices.

In [37]:
coeffdf.tail(10)

,Word,Coefficient
29063,worldwide,-0.388494
12825,hours,-0.397392
25952,system,-0.400174
26032,taliban,-0.402863
26329,terror,-0.417131
11911,hacking,-0.417332
6213,congo,-0.422793
17475,my,-0.462937
23041,sanctions,-0.524785
22838,run,-0.632614


But, these words do not seem to be correctly predicted as "hacking" and "taliban" are shown to have less impact on stock prices. Inorder to overcome these problems, we can try predicting with more than 1 word at the same time as sometimes a single word doesnt have any meaning on its own.

To achive this, we usse the n-gram model that takes the number of words as parameters.

In [38]:
newvectorizer = CountVectorizer(ngram_range=(2,2))
newtrain = newvectorizer.fit_transform(train_set)

In [39]:
print(newtrain.shape)

(1359, 318953)


In [40]:
newmodel=LogisticRegression(solver='lbfgs', max_iter=1000)
newmodel=newmodel.fit(newtrain,train['Label'])

Again, we repeat the process of creating the test set

In [41]:
test_set = []
for row in range(0,len(test.index)):
    test_set.append(' '.join(str(x) for x in test.iloc[row,2:27]))
newtest = newvectorizer.transform(test_set)
newpredictions = newmodel.predict(newtest)

We now print the corsstab to view the accuracy of our model

In [42]:
pd.crosstab(test["Label"], newpredictions, rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,112,187
1,86,245


Our accuracy now increased to 56 % thus showing a better reuslt.
We can again compare coefficients with the words to check the most impacting of them all

In [43]:
newwords = newvectorizer.get_feature_names()
newcoeffs = newmodel.coef_.tolist()[0]
newcoeffdf = pd.DataFrame({'Word' : newwords, 
                        'Coefficient' : newcoeffs})
newcoeffdf = newcoeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])

Lets print the first 10 values in the sorted list

In [44]:
newcoeffdf.head(10)

,Word,Coefficient
137057,in china,0.253284
236720,right to,0.250883
274604,the first,0.246985
21304,and other,0.239722
11958,after the,0.221675
165790,likely to,0.219781
119274,government has,0.213843
280479,this is,0.212799
126163,have to,0.208299
272706,that they,0.207601


This seems to be predicted correctly as weighted words are shown to have high impact

In [45]:
newcoeffdf.tail(10)

,Word,Coefficient
38680,been arrested,-0.185310
284274,to help,-0.185360
191406,nuclear weapons,-0.197722
134904,if he,-0.198955
273961,the country,-0.199566
248545,sexual abuse,-0.203907
296018,up in,-0.208763
284516,to kill,-0.212105
42331,bin laden,-0.220662
311993,with iran,-0.222995


Though this has high accuracy, still words like 'bin laden' are shown to have low impact. 
So, the only way to continue is keep checking for different n gram values.

In [66]:
newvectorizer3 = CountVectorizer(ngram_range=(3,3))
newtrain3 = newvectorizer3.fit_transform(train_set)

In [67]:
print(newtrain3.shape)

(1359, 519904)


In [68]:
newmodel3=LogisticRegression(solver='lbfgs', max_iter=1000)
newmodel3=newmodel3.fit(newtrain3,train['Label'])

Again, we repeat the process of creating the new test set

In [70]:
test_set = []
for row in range(0,len(test.index)):
    test_set.append(' '.join(str(x) for x in test.iloc[row,2:27]))
newtest3=newvectorizer3.transform(test_set)
newpredictions3=newmodel3.predict(newtest3)

In [71]:
pd.crosstab(test["Label"], newpredictions3, rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,6,293
1,9,322


The accuracy now decreased to 52 %

In [72]:
newwords3 = newvectorizer3.get_feature_names()
newcoeffs3 = newmodel3.coef_.tolist()[0]
newcoeffdf3 = pd.DataFrame({'Word' : newwords3, 
                        'Coefficient' : newcoeffs3})
newcoeffdf3 = newcoeffdf3.sort_values(['Coefficient', 'Word'], ascending=[0, 1])

In [73]:
newcoeffdf3.head(10)

,Word,Coefficient
311648,of the country,0.166861
465577,to the us,0.163057
204741,human rights watch,0.140899
439686,the right to,0.136982
294832,nobel peace prize,0.134304
320948,on the internet,0.121709
432751,the first time,0.121387
219759,in west bank,0.114705
448905,this is the,0.110555
11418,according to new,0.107022


In [74]:
newcoeffdf.tail(10)

,Word,Coefficient
38680,been arrested,-0.185310
284274,to help,-0.185360
191406,nuclear weapons,-0.197722
134904,if he,-0.198955
273961,the country,-0.199566
248545,sexual abuse,-0.203907
296018,up in,-0.208763
284516,to kill,-0.212105
42331,bin laden,-0.220662
311993,with iran,-0.222995


Thus, even though heavy impacting words are shown to have lower coefficients, the accuracy of our model is still the same

Hence we arrive at the conclusion that our model has helped bring the accuracy to 57 % at most, and we were successfull in predicting most of the headlines effect on stock prices.